In [31]:
%pip install --upgrade e2b_code_interpreter cohere==5.5.3

Note: you may need to restart the kernel to use updated packages.


In [32]:
# Instantiate the Cohere client

import cohere
import os
import sys
import json

import os
from dotenv import load_dotenv
load_dotenv()

COHERE_API_KEY = os.environ["COHERE_API_KEY"]
E2B_API_KEY = os.getenv("E2B_API_KEY")

co = cohere.Client(api_key=COHERE_API_KEY)

In [33]:
SYSTEM_PROMPT = """
## your job & context
you are a python data scientist. you are given tasks to complete and you run python code to solve them.
You DO NOT MAKE SYNTAX MISTAKES OR FORGET ANY IMPORTS
- the python code runs in jupyter notebook.
- every time you call `execute_python` tool, the python code is executed in a separate cell. it's okay to multiple calls to `execute_python`.
- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.
- you can run any python code you want, everything is running in a secure sandbox environment.
"""

USER_PROMPT = "Run a Python code that simulates outcomes of playing rocks paper scissors 50 times and based on that, tell what was the most lucky to play."

In [34]:
# Define the tools

def code_interpret(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.notebook.exec_cell(
    code,
    # Stream stdout and stderr from the Code Interpreter
    on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
    on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
    # You can also stream code execution results
    # on_result=...
  )

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec


execute_python_tool =   {
  "name": "execute_python",
  "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
  "parameter_definitions": {
      "code": {
          "description": "The python code to execute in a single cell.",
          "type": "str",
          "required": True
    }
  },
}


# helper function for routing to the correct tool
def invoke_tool(tool_call: cohere.ToolCall, e2b_code_interpreter):
  if tool_call.name == execute_python_tool["name"]:
    code = tool_call.parameters["code"]

    return [{
        "is_success": code_interpret(e2b_code_interpreter, code)  # TBD Fix
    }]
  else:
    raise f"Unknown tool name '{tool_call.name}'"

In [35]:
# Check what tools the model wants to use and how to use them
# Define chat with model

def chat(e2b_code_interpreter, user_message):
  print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

  res = co.chat(
      model="command-r-plus",
      preamble="Today is Thursday, may 23, 2024",
      message=USER_PROMPT,
      force_single_step=False,
      tools=[execute_python_tool])

  while res.tool_calls:
    print(res.text) # This will be an observation and a plan with next steps

    # invoke the recommended tools
    tool_results = []
    for call in res.tool_calls:
      # Add print statements to inspect the variables
      print()
      print(f"🚨 MONITORING call: {call}")
      print()
      print(f"🚨 invoke_tool(call): {invoke_tool(call, e2b_code_interpreter)}")
      print()

      tool_results.append({"call": call, "outputs": invoke_tool(call, e2b_code_interpreter)})

    # send back the tool results
    res = co.chat(
      model="command-r-plus",
      chat_history=res.chat_history,
      message="",
      force_single_step=False,
      tools=[execute_python_tool],
      tool_results=tool_results,
    )
  for choice in res.chat_history:   # TBD: Find correct attribute instead of choices... Is it chat_history?
    if choice.message.tool_calls and len(choice.message.tool_calls) > 0:
      for tool_call in choice.message.tool_calls:
        if tool_call.name == "execute_python":
          if "code" in tool_call.function.arguments:
            args = json.loads(tool_call.function.arguments)
            code = args["code"]          
            print("CODE TO RUN")
            print(code)
            execution_output = code_interpret(e2b_code_interpreter, code)
            return execution_output
    else:
      print("Answer:", choice.message.content)
  
  print(json.dumps(res, indent=2))
  

In [36]:
# Print the results

from e2b_code_interpreter import CodeInterpreter

with CodeInterpreter(api_key=E2B_API_KEY) as code_interpreter:
  execution_output = chat( # ERROR: str object has no attribute tool_calls
    code_interpreter,
    USER_PROMPT,
  )

  if execution_output == None:
    print("No output from the code interpreter. Did the LLM generate any code?")
    sys.exit(1)

  # Access stdout
  print("Stdout:", execution_output.logs.stdout)
  # Access stderr
  print("Stderr:", execution_output.logs.stderr)

  # Access any runtime errors
  print("AI-generated Python runtime error:", execution_output.error)

  # Access any results of code execution - charts, interpreter last line, images, etc.
  print("Results:", execution_output.results)
  if len(execution_output.results) == 0:
    sys.exit(0)

execution_output.results[0]


User Message: Run a Python code that simulates outcomes of playing rocks paper scissors 50 times and based on that, tell what was the most lucky to play.
I will write and execute Python code that simulates the outcomes of playing rock, paper, scissors 50 times and determine which one is the luckiest to play.

🚨 MONITORING call: name='execute_python' parameters={'code': 'import random\n\n# Define the three choices\nchoices = [\'rock\', \'paper\', \'scissors\']\n\n# Simulate playing rock, paper, scissors 50 times\nresults = []\nfor _ in range(50):\n    player1 = random.choice(choices)\n    player2 = random.choice(choices)\n    results.append((player1, player2))\n\n# Count the number of wins for each choice\nchoice_wins = {\n    \'rock\': 0,\n    \'paper\': 0,\n    \'scissors\': 0\n}\n\nfor player1, player2 in results:\n    if player1 == \'rock\' and player2 == \'scissors\' or (player1 == \'scissors\' and player2 == \'rock\'):\n        choice_wins[player1] += 1\n    elif player1 == \'pap

AttributeError: 'str' object has no attribute 'tool_calls'